In [ ]:
import utils
import cleaning
import pandas as pd


In [ ]:
cleaning.recipes_ratings_merged()
df = pd.read_csv("../data/Processed/recipes with ratings.csv")

In [ ]:
df.sample(5)

In [ ]:
# Check the first element of each column to determine if they are strings or lists
first_tags = df['tags'].iloc[0]
first_ingredients = df['ingredients'].iloc[0]
first_nutrition = df['nutrition'].iloc[0]
first_steps = df['steps'].iloc[0]

# Print the type of the first element of each column
types = {
    'tags': type(first_tags),
    'ingredients': type(first_ingredients),
    'nutrition': type(first_nutrition),
    'steps': type(first_steps)
}

# Display the types
for column, typ in types.items():
    print(f'The first element of the {column} column is of type: {typ}')

# Convert the 'tags' column from string to list
if isinstance(df['tags'].iloc[0], str):
    df['tags'] = df['tags'].apply(lambda x: x.strip('[]').replace("'", '').split(', '))

# Convert the 'ingredients' column from string to list
if isinstance(df['ingredients'].iloc[0], str):
    df['ingredients'] = df['ingredients'].apply(lambda x: x.strip('[]').replace("'", '').split(', '))

# Convert the 'nutrition' column from string to list
if isinstance(df['nutrition'].iloc[0], str):
    df['nutrition'] = df['nutrition'].apply(lambda x: x.strip('[]').replace("'", '').split(', '))

# Convert the 'steps' column from string to list
if isinstance(df['steps'].iloc[0], str):
    df['steps'] = df['steps'].apply(lambda x: x.strip('[]').replace("'", '').split(', '))

In [ ]:
# Function to categorize time into intervals
def categorize_time(minutes):
    if minutes > 90:
        return 'More than 1h 30min'
    else:
        interval = (minutes // 15) * 15
        return f'About {interval} min' if interval != 0 else 'About 15 min'

# Apply the function to create a new column
df['cookking_time'] = df['minutes'].apply(categorize_time)
df
